# Tobacco CNN

In [ ]:
import tensorflow as tf
import os
import gdown
import zipfile

In [ ]:
# Google Drive file ID
file_id = '1NHF-Cxboa16uSg7c265FufvHrTOvGr2N'

# Output file path
output_zip = '/content/Tobacco_Data.zip'

# Download the file from Google Drive
url = f'https://drive.google.com/uc?id={file_id}'
gdown.download(url, output_zip, quiet=False)

# Extract the zip file
with zipfile.ZipFile(output_zip, 'r') as zip_ref:
    zip_ref.extractall('/content')

# Define the base directory
base_dir = '/content/Tobacco_Data'

# Print the list of extracted files and folders
print('Files and folders in the extracted zip:')

Downloading...
From: https://drive.google.com/uc?id=1NHF-Cxboa16uSg7c265FufvHrTOvGr2N
To: /content/Tobacco_Data.zip
100%|██████████| 26.6M/26.6M [00:00<00:00, 125MB/s]


Files and folders in the extracted zip:


## Image Preprocessing

In [ ]:
IMAGE_SIZE = 224
BATCH_SIZE = 64

datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale = 1./255,
    validation_split = 0.2
)

train_generator = datagen.flow_from_directory(
  base_dir,
  target_size = (IMAGE_SIZE, IMAGE_SIZE),
  batch_size = BATCH_SIZE,
  subset = 'training'
)

val_generator = datagen.flow_from_directory(
  base_dir,
  target_size = (IMAGE_SIZE, IMAGE_SIZE),
  batch_size = BATCH_SIZE,
  subset = 'validation'
)

Found 347 images belonging to 3 classes.
Found 86 images belonging to 3 classes.


In [ ]:
print(train_generator.class_indices)
labels = '\n'.join(sorted(train_generator.class_indices.keys()))
with open('labels.txt', 'w') as f:
  f.write(labels)

{'Burley': 0, 'Dark Fired': 1, 'Flue Cured': 2}


In [ ]:
IMAGE_SHAPE = (IMAGE_SIZE, IMAGE_SIZE, 3)

base_model = tf.keras.applications.MobileNetV2(input_shape=IMAGE_SHAPE, include_top=False, weights='imagenet')

9406464/9406464 [==============================] - 0s 0us/step


In [ ]:
base_model.trainable = False

In [ ]:
model = tf.keras.Sequential([
    base_model,
    tf.keras.layers.Conv2D(32, 3, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dense(3, activation='softmax')
])

In [ ]:
model.compile(optimizer=tf.keras.optimizers.Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
epochs = 10

history = model.fit(train_generator, epochs=epochs, validation_data=val_generator)

Epoch 1/10
6/6 [==============================] - 39s 6s/step - loss: 1.6991 - accuracy: 0.4928 - val_loss: 0.8895 - val_accuracy: 0.6163
Epoch 2/10
6/6 [==============================] - 37s 5s/step - loss: 0.7322 - accuracy: 0.6484 - val_loss: 0.6117 - val_accuracy: 0.6860
Epoch 3/10
6/6 [==============================] - 28s 5s/step - loss: 0.4089 - accuracy: 0.8386 - val_loss: 0.3213 - val_accuracy: 0.8605
Epoch 4/10
6/6 [==============================] - 27s 5s/step - loss: 0.1906 - accuracy: 0.9308 - val_loss: 0.2350 - val_accuracy: 0.8953
Epoch 5/10
6/6 [==============================] - 30s 5s/step - loss: 0.0973 - accuracy: 0.9654 - val_loss: 0.1382 - val_accuracy: 0.9419
Epoch 6/10
6/6 [==============================] - 24s 4s/step - loss: 0.0374 - accuracy: 0.9942 - val_loss: 0.1449 - val_accuracy: 0.9651
Epoch 7/10
6/6 [==============================] - 25s 4s/step - loss: 0.0277 - accuracy: 0.9971 - val_loss: 0.1860 - val_accuracy: 0.9302
Epoch 8/10
6/6 [==================

In [ ]:
saved_model_dir = ''
tf.saved_model.save(model, saved_model_dir)

converter = tf.lite.TFLiteConverter.from_saved_model(saved_model_dir)
tflite_model = converter.convert()

with open('model_tflite', 'wb') as f:
  f.write(tflite_model)